## Hybrid Oral-Cancer Biomarker Dataset Builder — Notebook Guide

1) Purpose

This notebook takes biomarker statistics from Excel/PDF sources, normalizes them (fluids/classes), deduplicates by strength of evidence, and simulates a row-level hybrid dataset with realistic patient-like values. It then harmonizes columns against a target biomarker map and provides a validation checker to compare simulated aggregates with the source reference stats.

2) Pipeline at a Glance
	1.	Normalize inputs
	•	Map raw fluid strings to {blood, saliva, urine, other}
	•	Map class strings to {OSCC, Benign Lesion, Healthy Control}
	2.	Parse numerics from Excel
	•	Robust extraction of Mean, SD, n from free-form cells
	3.	Seed with PDF/Manual rows
	•	Add extra study rows when only present in PDFs
	4.	Merge + Deduplicate
	•	Keep the record with largest n; if tie, keep latest study year
	5.	Emit reference table → hybrid_reference_stats.csv
	6.	Strict simulation
	•	For each (Marker, Fluid, Class) with (mean, sd) present, draw n values
	•	Clip outliers (0..mean+3*sd), attach demographics + stage
	7.	Column harmonization
	•	Ensure all biomarkers in biomarker_map exist as columns; preserve dataset columns
	8.	Validation
	•	Compare dataset aggregates to the reference stats (n/mean/sd checks)

3) File Outputs
	•	hybrid_reference_stats.csv — deduped study-level stats used as ground truth
	•	hybrid_reference_grouped_strict.csv — simulated patient-like rows
	•	skipped_marker_diagnosis_pairs.csv — pairs skipped due to missing mean/sd
	•	Hybrid_Dataset.csv — column-harmonized dataset (adds any missing biomarkers)
	•	added_columns_report.csv — list of biomarkers added as empty columns

4) Assumptions & Notes
	•	The Excel has columns similar to: Biomarker | Marker | Fluid | Class | Mean | SD | n | Source_Study
	•	Classes in PDFs may use variants (e.g., “Control”, “Benign (….)”) and are normalized.
	•	Simulation requires both Mean and SD. If missing either, the pair is skipped.

5) Reproducibility
	•	A single np.random.seed(42) is used in the simulator for deterministic results.

6) Extend/Customize
	•	Add more PDF tuples in pdf_stats as needed.
	•	Tweak clipping/priors for demographics or add additional metadata columns.
	•	Update biomarker_map to your canonical set; ordering is honored in the output.

In [ ]:
import pandas as pd
import numpy as np
import re

# ---- 0. FLUID & CLASS NORMALIZATION ----
# The following helpers standardize heterogeneous strings for fluid types and classes
# so downstream logic can rely on a small, clean set of categories.

def normalize_fluid(fluid_raw):
    """Standardize fluid type across all inputs."""
    # Coerce to lowercase string and trim whitespace to avoid case/format inconsistencies.
    raw = str(fluid_raw).strip().lower()
    # Treat 'plasma' and 'serum' as blood to keep analysis consistent across studies.
    if raw in ['plasma', 'serum']:
        return 'blood'
    # Pass through known labels unchanged; anything else falls back to 'other'.
    elif raw in ['blood', 'saliva', 'urine']:
        return raw
    # Handle empty/NaN-like values explicitly.
    elif not raw or raw in ['nan', '']:
        return 'other'
    else:
        return 'other'

def map_class(cls_raw):
    """Standardize class/category names."""
    # Normalize input to lowercase for robust keyword matching.
    c = str(cls_raw).strip().lower()
    # Anything suggestive of OSCC/malignancy maps to canonical label 'OSCC'.
    if any(x in c for x in ['oscc', 'cancer', 'malig']):
        return 'OSCC'
    # Premalignant/benign variants map to 'Benign Lesion'.
    elif any(x in c for x in ['premalign', 'benign']):
        return 'Benign Lesion'
    # Controls/healthy variants map to 'Healthy Control'.
    elif any(x in c for x in ['control', 'healthy']):
        return 'Healthy Control'
    # Unknown/unmappable classes are dropped by returning None.
    return None

# ---- 1. EXCEL NUMERIC EXTRACTION ----
# Free-form Excel cells may contain text like "12.3 ± 1.4 (n=30)".
# get_numeric() extracts the first numeric token (int/float) robustly.

def get_numeric(val):
    """Extract numeric value from cell."""
    if pd.isnull(val):
        return None
    # If it's already numeric, return as-is.
    if isinstance(val, (int, float)):
        return val
    # Remove thousands separators, then match float-or-int pattern.
    m = re.search(r'[-+]?\d*\.\d+|\d+', str(val).replace(',', ''))
    return float(m.group()) if m else None

# ---- 2. LOAD EXCEL DATA ----
# Read all sheets, normalize headers, then pull rows where Mean/SD/n are present.
# Map Biomarker/Marker naming, normalize Fluid and Class, and collect standardized rows.

excel_path = 'biomarkers papaer data final.xlsx'
excel_stats = []

for sheet in pd.ExcelFile(excel_path).sheet_names:
    df = pd.read_excel(excel_path, sheet_name=sheet)
    # Strip whitespace from header names to avoid key mismatches.
    df.columns = df.columns.str.strip()
    # Iterate row-wise to extract numeric fields and normalized metadata.
    for _, row in df.iterrows():
        mean = get_numeric(row.get('Mean'))
        sd = get_numeric(row.get('SD'))
        n = get_numeric(row.get('n'))
        # Only keep rows that have all three numeric pillars for simulation/validation.
        if mean is not None and sd is not None and n is not None:
            # Prefer 'Biomarker' column; fall back to 'Marker' if absent.
            biomarker = str(row.get('Biomarker', row.get('Marker', ''))).strip()
            fluid = normalize_fluid(row.get('Fluid', ''))
            cls_out = map_class(row.get('Class', ''))
            # Drop rows whose class couldn't be mapped to the canonical triad.
            if not cls_out:
                continue
            # Accumulate a normalized stats row for later merging/deduplication.
            excel_stats.append({
                'Source_Study': str(row.get('Source_Study', '')).strip(),
                'Fluid': fluid,
                'Marker': biomarker,
                'Class': cls_out,
                'Mean': mean,
                'SD': sd,
                'n': int(n)
            })

# ---- 3. PDF DATA (ADD MORE AS NEEDED) ----
# Manual entries from PDFs or papers not captured in the Excel sheets.
# Keep values None where mean/sd are unavailable; the simulator will skip those.

pdf_stats = [
    # (Source_Study, Fluid, Marker, Class, Mean, SD, n)
    ("Kedia et al., 2023", "saliva", "IL6", "OSCC", 45.3, None, 50),
    ("Kedia et al., 2023", "saliva", "IL6", "Control", 12.7, None, 50),
    ("Kedia et al., 2023", "saliva", "miR21", "OSCC", None, None, 50),
    ("Kedia et al., 2023", "saliva", "miR21", "Control", None, None, 50),
    ("Kedia et al., 2023", "saliva", "p16_Methylation", "OSCC", None, None, 50),
    ("Kedia et al., 2023", "saliva", "p16_Methylation", "Control", None, None, 50),
    ("Li et al., 2004", "saliva", "IL8", "OSCC", None, None, 32),
    ("Li et al., 2004", "saliva", "IL8", "Control", None, None, 32),
    ("Alzoubi et al., 2025", "saliva", "IL6", "OSCC", 250, 25, 30),
    ("Alzoubi et al., 2025", "saliva", "IL6", "Benign (oral premalignant lesion)", 100, 15, 30),
    ("Alzoubi et al., 2025", "saliva", "IL6", "Control", 30, 5, 30),
    ("Alzoubi et al., 2025", "saliva", "MMP9", "OSCC", 500, 50, 30),
    ("Alzoubi et al., 2025", "saliva", "MMP9", "Benign (oral premalignant lesion)", 300, 40, 30),
    ("Alzoubi et al., 2025", "saliva", "MMP9", "Control", 100, 20, 30),
    ("Alzoubi et al., 2025", "saliva", "LDH", "OSCC", 850, 70, 30),
    ("Alzoubi et al., 2025", "saliva", "LDH", "Benign (oral premalignant lesion)", 600, 60, 30),
    ("Alzoubi et al., 2025", "saliva", "LDH", "Control", 300, 50, 30),
    ("Gleber-Netto et al., 2016", "saliva", "IL8", "OSCC", None, None, 60),
    ("Gleber-Netto et al., 2016", "saliva", "IL8", "Benign (potentially malignant oral disorder)", None, None, 60),
    ("Gleber-Netto et al., 2016", "saliva", "IL8", "Control", None, None, 60),
    ("Gleber-Netto et al., 2016", "saliva", "IL8", "OSCC", None, None, 35),
    ("Brinkmann et al., 2011", "saliva", "IL8", "Control", None, None, 51),
]

# ---- 4. MERGE ALL, NORMALIZE FLUID/CLASS, DEDUP ----
# Merge Excel + PDF rows, normalize, then deduplicate by (Fluid, Marker, Class).
# Resolution rule: keep the row with highest n; if tie, keep the most recent study year.

def get_year(src):
    # Extract a 4-digit year from the 'Source_Study' string, default 0 if absent.
    m = re.search(r'(\d{4})', src)
    return int(m.group(1)) if m else 0

all_stats = []

# From Excel (already normalized)
for d in excel_stats:
    all_stats.append(d)

# From PDF (normalize class/fluid on ingest)
for (src, fluid, marker, cls, mean, sd, n) in pdf_stats:
    fluid_norm = normalize_fluid(fluid)
    cls_norm = map_class(cls)
    if cls_norm:
        all_stats.append({
            'Source_Study': src, 'Fluid': fluid_norm, 'Marker': marker, 'Class': cls_norm,
            'Mean': mean, 'SD': sd, 'n': n
        })

# Deduplicate: keep highest n, if tie, keep latest study year
best_stats = {}
for d in all_stats:
    key = (d['Fluid'], d['Marker'], d['Class'])
    if key not in best_stats or d['n'] > best_stats[key]['n'] or (
        d['n'] == best_stats[key]['n'] and get_year(d['Source_Study']) > get_year(best_stats[key]['Source_Study'])
    ):
        best_stats[key] = d
final_stats = list(best_stats.values())

# ---- 5. SAVE OUTPUT ----
# Emit the canonical reference stats table used by the simulator and validator.

refstats_df = pd.DataFrame(final_stats)
refstats_df = refstats_df[['Source_Study', 'Fluid', 'Marker', 'Class', 'Mean', 'SD', 'n']]
refstats_df.to_csv("hybrid_reference_stats.csv", index=False)
print(f"Reference stats saved to hybrid_reference_stats.csv with {len(refstats_df)} rows.")

def strict_hybrid_simulation(
    ref_stats_csv,
    filename="hybrid_reference_grouped_strict.csv",
    skipped_filename="skipped_marker_diagnosis_pairs.csv"
):
    # Fixed seed for reproducibility of simulated values.
    np.random.seed(42)  # 🔒 Reproducibility

    def normalize_fluid(f):
        # Local capitalized mapping for column naming consistency (Blood/Saliva/Urine).
        f = str(f).strip().lower()
        return 'Blood' if f in ['plasma', 'serum', 'blood'] else f.capitalize()

    # 📥 Load reference stats
    ref = pd.read_csv(ref_stats_csv)
    # Define the complete set of biomarker+fluid columns seen in reference.
    all_marker_fluid = sorted({
        f"{r['Marker']}_{normalize_fluid(r['Fluid'])}" for _, r in ref.iterrows()
    })
    # Meta columns to enrich each simulated row with demographics/stage.
    meta_cols = ['Patient_ID', 'Diagnosis', 'Age', 'Gender', 'Smoking_Status', 'Cancer_Stage']
    all_rows = []
    skipped = []
    pid_counter = 1

    # Iterate over each reference row and sample n values when mean/sd are available.
    for _, row in ref.iterrows():
        marker_col = f"{row['Marker']}_{normalize_fluid(row['Fluid'])}"
        diagnosis = row['Class']
        n = int(row['n'])
        mean = row['Mean']
        sd = row['SD']

        if pd.notnull(mean) and pd.notnull(sd):
            for _ in range(n):
                # Initialize an empty row with NaNs for all biomarkers and meta fields.
                sim_row = {col: np.nan for col in all_marker_fluid + meta_cols}
                # Deterministic identifier increases monotonically.
                sim_row['Patient_ID'] = f"P{pid_counter:05d}"
                sim_row['Diagnosis'] = diagnosis
                # Lightly realistic demographics with simple priors.
                sim_row['Age'] = np.random.randint(30, 80)
                sim_row['Gender'] = np.random.choice(['Male', 'Female'], p=[0.6, 0.4])
                sim_row['Smoking_Status'] = np.random.choice(['Current', 'Former', 'Never'], p=[0.3, 0.3, 0.4])
                sim_row['Cancer_Stage'] = (
                    np.random.choice(['Stage I', 'Stage II', 'Stage III', 'Stage IV']) if diagnosis == "OSCC" else "N/A"
                )

                # ✂️ Clip to avoid unrealistic outliers while preserving central tendency.
                sampled = np.random.normal(loc=mean, scale=sd)
                val = np.clip(sampled, 0, mean + 3 * sd)
                sim_row[marker_col] = val

                all_rows.append(sim_row)
                pid_counter += 1
        else:
            # Keep track of marker/class pairs we could not simulate (due to missing mean/sd).
            skipped.append({'Marker_Fluid': marker_col, 'Diagnosis': diagnosis})
            print(f"⚠️ Skipping: {marker_col} for {diagnosis} — missing mean or SD")

    # 📊 Final dataframe assembled in deterministic biomarker order + meta columns at end.
    df = pd.DataFrame(all_rows, columns=all_marker_fluid + meta_cols)
    df.to_csv(filename, index=False)
    print(f"\n✅ Dataset saved to: {filename} with {len(df)} rows.")

    # 📤 Save skipped entries if any to aid future data collection/completion.
    if skipped:
        skipped_df = pd.DataFrame(skipped).drop_duplicates()
        skipped_df.to_csv(skipped_filename, index=False)
        print(f"⚠️ {len(skipped_df)} marker-diagnosis pairs skipped and saved to: {skipped_filename}")
        return df, skipped_df

    return df, None
    
# Usage:
# Run the simulator against the just-created reference file to produce row-level data.
df, skipped_df = strict_hybrid_simulation(
    "hybrid_reference_stats.csv",
    "hybrid_reference_grouped_strict.csv",
    "skipped_marker_diagnosis_pairs.csv"
)

# -------------------------------------------------------------------
# Column harmonization: ensure the dataset matches a canonical map.
# -------------------------------------------------------------------

biomarker_map = {
    'ctDNA_Blood': ['blood'],
    'miR21_Blood': ['blood'], 'miR21_Saliva': ['saliva'],
    'miR31_Blood': ['blood'], 'miR31_Saliva': ['saliva'],
    'miR184_Saliva': ['saliva'],
    'SCCAg_Blood': ['blood'],
    'Cyfra211_Blood': ['blood'],
    'IL6_Blood': ['blood'], 'IL6_Saliva': ['saliva'],
    'IL8_Blood': ['blood'], 'IL8_Saliva': ['saliva'],
    'TNFa_Blood': ['blood'], 'TNFa_Saliva': ['saliva'],
    'CRP_Blood': ['blood'],
    'LDH_Blood': ['blood'], 'LDH_Saliva': ['saliva'],
    'ExosomalRNA_Saliva': ['saliva'],
    'MDA_Blood': ['blood'], 'MDA_Saliva': ['saliva'],
    'OHdG_Blood': ['blood'], 'OHdG_Saliva': ['saliva'],
    'NO_Blood': ['blood'], 'NO_Saliva': ['saliva'],
    'MMP2_Blood': ['blood'], 'MMP2_Saliva': ['saliva'],
    'MMP9_Blood': ['blood'], 'MMP9_Saliva': ['saliva'],
    'p16_Methylation_Saliva': ['saliva'],
    'GSH_Blood': ['blood'], 'GSH_Saliva': ['saliva'],
    'SOD_Blood': ['blood'], 'SOD_Saliva': ['saliva'],
    'CAT_Blood': ['blood'],
    'VitaminD_Blood': ['blood'],
    'Ferritin_Blood': ['blood'],
    'TAC_Blood': ['blood'], 'TAC_Saliva': ['saliva'],
    'Fibrinogen_Blood': ['blood'],
    'UricAcid_Blood': ['blood'], 'UricAcid_Saliva': ['saliva'],
    'HA_Blood': ['blood'], 'HA_Saliva': ['saliva'],
    'PGE2_Blood': ['blood'], 'PGE2_Saliva': ['saliva'],
    'B2M_Blood': ['blood'],
    'Cortisol_Saliva': ['saliva'],
    'Galectin3_Blood': ['blood'], 'Galectin3_Saliva': ['saliva'],
    'YKL40_Blood': ['blood'],
    'VEGF_Blood': ['blood'], 'VEGF_Saliva': ['saliva'],
    'S100A7_Saliva': ['saliva'],
    'Periostin_Blood': ['blood'], 'Periostin_Saliva': ['saliva'],
    'LCN2_Blood': ['blood'], 'LCN2_Saliva': ['saliva'],
    'Ceruloplasmin_Blood': ['blood'],
    'LipidPerox_Blood': ['blood'], 'LipidPerox_Saliva': ['saliva'],
    'Anti_p53_Blood': ['blood'],
    'Anti_MAGE_Blood': ['blood'],
    'lncRNA_Blood': ['blood'], 'lncRNA_Saliva': ['saliva'],
    'hTERT_Blood': ['blood'],
    'Zinc_Blood': ['blood'], 'Zinc_Saliva': ['saliva'],
    'Copper_Blood': ['blood'],
    'Selenium_Blood': ['blood']
}
# Canonical ordered list for final column arrangement.
biomarker_names = list(biomarker_map.keys())

# Load your hybrid dataset (update file path as needed)
df = pd.read_csv('hybrid_reference_grouped_strict.csv')

# ---- 2. CHECK AND ADD MISSING COLUMNS ----
# Create any missing biomarker columns to meet the canonical schema (filled with NaN).

missing_cols = [col for col in biomarker_names if col not in df.columns]

for col in missing_cols:
    df[col] = np.nan

# ---- 3. REORDER COLUMNS: Put biomarker columns in specified order, keep other columns at the end ----
# Preserve any non-biomarker columns (meta fields) after the canonical biomarker list.

other_cols = [c for c in df.columns if c not in biomarker_names]
final_cols = biomarker_names + [c for c in other_cols if c not in biomarker_names]
df = df[final_cols]

# ---- 4. OUTPUT ----
# Persist the harmonized dataset and a simple report of columns that were added.

# Save the corrected file if needed
corrected_filename = 'Hybrid_Dataset.csv'
df.to_csv(corrected_filename, index=False)

# Save to CSV
missing_cols_df = pd.DataFrame(missing_cols, columns=['Added_Columns'])
missing_cols_df.to_csv('added_columns_report.csv', index=False)

print("✅ Missing columns report saved as 'added_columns_report.csv'")
print(f"\nTotal columns added: {len(missing_cols)}")
print(f"Corrected dataset saved to: {corrected_filename}")

# ------------------------------------------------------------
# Validation helper: compare dataset aggregates to references.
# ------------------------------------------------------------

def validate_against_reference(ref_stats_csv, hybrid_csv):
    ref = pd.read_csv(ref_stats_csv)
    hybrid = pd.read_csv(hybrid_csv)
    results = []

    for _, row in ref.iterrows():
        # Build the column name used in the dataset for this marker+fluid pair.
        marker_col = f"{row['Marker']}_{row['Fluid'].capitalize()}"
        diagnosis = row['Class']
        n_ref = int(row['n'])
        mean_ref = row['Mean']
        sd_ref = row['SD']
        # Filter to rows with the same class and collect non-null values for the biomarker.
        vals = hybrid.loc[hybrid['Diagnosis'] == diagnosis, marker_col].dropna()
        n_ds = len(vals)
        # Compute dataset mean/sd with ddof=1 (sample std) when applicable.
        mean_ds = np.round(vals.mean(), 4) if n_ds else None
        sd_ds = np.round(vals.std(ddof=1), 4) if n_ds > 1 else None
        # Tolerance checks: n must match exactly; mean within 5%; sd within 15% if present.
        results.append({
            'Marker_Fluid': marker_col, 'Diagnosis': diagnosis,
            'n_ref': n_ref, 'n_ds': n_ds,
            'mean_ref': mean_ref, 'mean_ds': mean_ds,
            'sd_ref': sd_ref, 'sd_ds': sd_ds,
            'n_ok': n_ds == n_ref,
            'mean_ok': mean_ds is not None and abs(mean_ds - mean_ref) < 0.05 * abs(mean_ref) if mean_ref else None,
            'sd_ok': sd_ds is not None and abs(sd_ds - sd_ref) < 0.15 * abs(sd_ref) if sd_ref else None,
        })
    return pd.DataFrame(results)

# Print the validation DataFrame for quick inspection.
print(validate_against_reference("hybrid_reference_stats.csv", "Hybrid_Dataset.csv"))

Reference stats saved to hybrid_reference_stats.csv with 51 rows.
